# Francisco Javier Concejo Iglesias
# Análisis de información no estructurada
# Actividad 2 Clasificación

In [1]:
#!python -m spacy download en_core_web_lg 

In [2]:
import pandas as pd
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
import spacy
import warnings
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np

#spacy.cli.download("en_core_web_sm")
nlp= spacy.load('en_core_web_lg')

english_stemmer = nltk.stem.SnowballStemmer('english')

from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2 style="text-align: center;">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

## 1. Lea el contenido del fichero csv en un DataFrame. 

In [3]:
data=pd.read_csv('C:/Users/javoc/OneDrive/Escritorio/MASTER/AnalisisDeInformacionNoEstructurada/Practica2/spam.csv')
print(data.shape)
data.head(1)

(5572, 2)


,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."


Nuestros datos están compuestos por 5572 filas y 2 columnas. Las filas representan los comentarios tanto de spam como de no spam. La primera columna es la categoría de el comentario y la segunda el propio comentario. La categoría puede tomar dos valores: spam si es un mensaje de spam y ham si es un mensaje de un usuario.

## 2. Realice el pre-procesamiento que considere necesario. Puede utilizar funciones de la librería NLTK o spaCy, a su voluntad. Recomendamos una escritura modular del código, para poder hacer pruebas posteriormente, y contestar a las preguntas del punto 6.

#### Eliminación de stopwords

In [4]:
stopwords = nlp.Defaults.stop_words
datos_stopwords=list(data["text"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)])))

#### Extracción de las formas canónicas

In [5]:
lista=list(data['text'])
formas_canonicas=[]
for x in lista:
    element=nlp(x)
    x=''
    for z in element:
        x+=' '+z.lemma_.lower()
    formas_canonicas.append(x)

El pre-procesamiento del texto para esta práctica va a consistir en dos partes. En la primera de ellas eliminamos las palabras vacías (stopwords) que carecen de significado. La segunda va a consistir en la extracción de la forma canónica de cada una de las palabras de nuestro corpus.

## 3. Divida el conjunto de documentos en un subconjunto de entrenamiento y otro de evaluación.


In [6]:
train, test = temporal_train_test_split(y = data, train_size=0.8)
print('Train:', train.shape)
print('Test:', test.shape)

Train: (4457, 2)
Test: (1115, 2)


## 4. Convierta el corpus de documentos en una matriz TF-idf.

In [7]:
warnings.filterwarnings("ignore")
def tfidf(datos):
    TFIDF_vectorizer = TfidfVectorizer()
    TFIDF_array=TFIDF_vectorizer.fit_transform(datos).toarray()
    vocab_TFIDF = TFIDF_vectorizer.get_feature_names()
    TFIDF=pd.DataFrame(TFIDF_array, columns=vocab_TFIDF)
    return(TFIDF).head(5)

tfidf(lista)

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,û_,û_thanks,ûªm,ûªt,ûªve,ûï,ûïharry,ûò,ûówell,žö
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


La matriz tfidf resuelve el principal problema que tiene la BOW de quitar importancia a aquellos términos que son menos frecuentes pero que pueden ser relevantes. Para resolver dicho problema, realiza una normalización.

Realizamos la matriz tfidf y observamos que hay muchos términos (columnas) los cuales, a priori, carecen de significado o contienen caracteres extraños. Sin embargo, no he decidido aplicar el parámetro "max_features" ya que este lo aplicaremos posteriormente en el ejercicio 6. Por lo tanto, podemos concluir que es necesario realizar un pre-procesamiento ya que no podemos obtener información de esta matriz.

## 5. Llegados a este punto, realice modelos de entrenamiento al menos con algoritmos de clasificador bayesiano ingenuo, máquinas SVM y un modelo basado en árbol de decisión. Obtenga resultados de accuracy de la clasificación, así como las matrices de confusión para los tres modelos.


In [8]:
texto = data.iloc[:,1]
label= data.iloc[:,0]

vectorizer = TfidfVectorizer()
texto_tfidf = vectorizer.fit_transform(texto)
texto_tfidf_dense = texto_tfidf.todense()

texto_train, texto_test, label_train, label_test = temporal_train_test_split(texto_tfidf, label, train_size = 0.8)

#### Clasificador bayesiano

In [9]:
bayesiano = MultinomialNB()
bayesiano.fit(texto_train,label_train)
label_pred_bayesiano=bayesiano.predict(texto_test)
matriz_confusión_bayesiano = pd.DataFrame(confusion_matrix(label_test,label_pred_bayesiano))
matriz_confusión_bayesiano.columns =['No spam', 'Spam']
matriz_confusión_bayesiano.index=['Predicción no spam','Predicción spam']

#### SVM

In [10]:
svm = SVC(kernel='linear')
svm.fit(texto_train, label_train)
label_pred_svm = svm.predict(texto_test)
matriz_confusión_svm = pd.DataFrame(confusion_matrix(label_test, label_pred_svm))
matriz_confusión_svm.columns =['No spam', 'Spam']
matriz_confusión_svm.index=['Predicción no spam','Predicción spam']

#### Arbol de decisión

In [11]:
arbol = tree.DecisionTreeClassifier()
arbol.fit(texto_train,label_train)
label_pred_arbol = arbol.predict(texto_test)
matriz_confusión_arbol = pd.DataFrame(confusion_matrix(label_test, label_pred_arbol))
matriz_confusión_arbol.columns =['No Spam', 'Spam']
matriz_confusión_arbol.index=['Predicción no spam','Predicción spam']

#### Matrices de confusión

In [12]:
display_side_by_side(matriz_confusión_bayesiano,matriz_confusión_svm,matriz_confusión_arbol, titles=['Bayesiano','SVM','Árbol de decisión'])

,No spam,Spam
Predicción no spam,970,0
Predicción spam,41,104
,No spam,Spam
Predicción no spam,966,4
Predicción spam,17,128
,No Spam,Spam
Predicción no spam,957,13
Predicción spam,22,123


He decidido realizar los tres modelos de clasificación supervisada para poder obtener una mayor comparación. Antes de analizar los resultados de las matrices de confusión, me gustaría realizar una breve explicación del código. En primer lugar, segmentamos nuestros datos en dos variables, una la cual contiene los textos en sí y otra que nos indica la categoría para cada mensaje. En segundo lugar, dividimos cada una de las variables en una muestra de entrenamiento sobre la cual entrenamos nuestro modelo y una muestra de validación sobre la cual lo validamos. Posteriormente, realizamos los tres modelos los cuales siguen una misma estuctura:
- Definición del modelo
- Entrenamiento del modelo
- Predicciones del modelo
- Creación de la matriz de confusión en base a estas predicciones y los valores reales

Como se puede observar, el modelo de clasificación bayesiana es más preciso a la hora de predecir los mensajes de spam que de verdad son spam. Por otra parte, el modelo SVM es el mejor a la hora de predecir los mensajes que realmente no son spam. Es importante destacar que los falsos positivos de nuestras matrices de confusión van a tener un impacto más negativo que los falsos negativos. Esto se debe a que los falsos positivos se refieren a aquellos mensajes que nuestro modelo detecta como spam pero en realidad no lo son. En caso de que, por ejemplo, estemos analizando la satisfacción de los usuarios en base a los comentarios, estos se ignorarían ya que nuestro modelo estaría indicando que son spam y por lo tanto no se tendría en cuenta información de usuarios reales que podría ser relevante. Por lo tanto, en términos de falsos positivos el clasificador bayesiano sería el mejor ya que no tiene. Pese a que ninguno de los modelos tiene el mayor número de verdaderos positivos y el menor de verdaderos negativos, no se puede concluir a simple vista cual es el mejor. Es por eso por lo que calculamos el accuracy para los 3 modelos.

#### Accuracy

In [13]:
def accuracy(df):
    TP=df.iloc[0,0]
    TN=df.iloc[1,1]
    FP=df.iloc[0,1]
    FN=df.iloc[1,0]
    acc=(TP+TN)/(TP+FN+FP+TN)
    return(acc)

print('El accuracy del clasificador bayesiano es de', accuracy(matriz_confusión_bayesiano).round(2))
print('El accuracy del SVM es de', accuracy(matriz_confusión_svm).round(2))
print('El accuracy del arbol de decisión es de', accuracy(matriz_confusión_arbol).round(2))

El accuracy del clasificador bayesiano es de 0.96
El accuracy del SVM es de 0.98
El accuracy del arbol de decisión es de 0.97


El término accuracy hace referencia a lo cerca que está el resultado de una medición del valor verdadero. Estadísticamente, está relacionada con el sesgo de una estimación. Para su cálculo, se dividirán los resultados verdaderos (tanto verdaderos positivos (VP) como verdaderos negativos (VN)) entre el número total de casos examinados (verdaderos positivos, falsos positivos, verdaderos negativos, falsos negativos). Como se puede observar, el modelo con mayor cantidad de predicciones positivas que fueron correctas (mayor exactitud) es el SVM, seguido del arbol de decisión y del clasificador bayesiano.

## 6. (Pregunta convalidable por la actividad de topic modelling. Confirmar con la profesora si está convalidado). Conteste a las siguientes preguntas basándote en evidencias de código. ¿Tiene influencia en el resultado final el número máximo de features a utilizar? ¿Modifica el resultado si se eliminan las stop words? ¿Y si se utilizan las formas canónicas?

#### Sin max features

In [14]:
def tfidf_max_feature(datos):
    TFIDF_vectorizer = TfidfVectorizer(max_features=20)
    TFIDF_array=TFIDF_vectorizer.fit_transform(datos).toarray()
    vocab_TFIDF = TFIDF_vectorizer.get_feature_names()
    TFIDF=pd.DataFrame(TFIDF_array, columns=vocab_TFIDF)
    return(TFIDF.head(5))

tfidf(lista)

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,û_,û_thanks,ûªm,ûªt,ûªve,ûï,ûïharry,ûò,ûówell,žö
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Con max features

In [15]:
tfidf_max_feature(lista)

,and,are,call,can,for,have,in,is,it,me,my,now,of,on,so,that,the,to,you,your
0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.406394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.913698,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0


Si aplicamos un número máximo de "features" se puede observar como obtenemos una matriz que nos indica la frecuencia de términos mas familiares que la matriz original la cual contenía términos sin sentido. Aquí podemos observar palabras conocidas las cuales la gran mayoría son preposiciones, artículos o pronombres. Llama la atención cómo la palabra "in" o "to" tienen una frecuencia de 1. A priori, esta matriz no nos proporciona gran información. 

Es importante destacar que sólo aparecen las primeras 5 filas ya que he aplicado la función "head" a la función que nos devuelve esta matriz para tener una presentación más limpia. Es por eso por lo que la gran mayoría de las frecuencias es de 0, ya que sólo aparecen los 5 primeros documentos del corpus. Si representásemos la matriz entera, estas frecuencias aparecerían más altas en otras filas de la matriz.

#### Sin stopwords

In [16]:
tfidf_max_feature(datos_stopwords)

,call,come,day,free,good,got,gt,it,know,like,ll,love,lt,me,no,now,ok,time,ur,you
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Si eliminamos las stop words, observamos cómo la matriz tfidf nos muestra la frecuencia de términos más comunes. Se pueden observar por lo general palabras de caracter mas positivo como "good" o "love". Llama la atención la columna que contiene el término "ll". Este es la abreviación del verbo "will". 

#### Formas canónicas

In [17]:
tfidf_max_feature(formas_canonicas)

,and,be,call,can,do,for,get,go,have,in,it,my,not,of,that,the,to,will,you,your
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.588717,0.605517,0.0,0.535501,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.410273,0.0,0.0,0.000000,0.0,0.0,0.0,0.911963,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.508159,0.0,0.000000,0.590192,0.0,0.000000,0.0,0.0,0.493849,0.0,0.0,0.0,0.386732,0.0,0.0,0.0


Si realizamos la matriz con las formas canónicas también vemos una gran mejoría en el resultado ya que se puede observar palabras conocidas.

#### Sin stopwords y con formas canónicas

In [18]:
df_formas_canonicas=pd.DataFrame(formas_canonicas)
formas_canonicas = df_formas_canonicas[0].squeeze()
canonicas_stopwords=list(formas_canonicas.apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)])))
tfidf_max_feature(canonicas_stopwords)

,come,day,free,good,gt,home,know,like,love,lt,need,ok,send,tell,text,think,time,txt,ur,want
0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.55623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.57546,0.0,0.0,0.599544,0.0,0.0
3,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,1.0,0.0,0.000000,0.0,0.0


Si realizamos la matrif tfidf sin tener en cuenta las stopwords y con las formas canónicas, el resultado en comparación con la matriz original es mucho mas claro y se puede obtener mas valor. Sin embargo, no se puede obtener mucha más información con respecto a la matriz tfidf que no tiene en cuenta las stopwords ya que estas son muy similares.

La conclusión final es que el pre-procesamiento realizado en el ejercicio 2 mejora mucho los resultados de la matriz tfidf. Hemos realizado 3 tipos de matrices tfidf:

1. Sin stopwords
2. Contando solo las formas canonicas
3. Sin stopwords y contando solo las formas canonicas

La tercera es la que mas información valiosa aporta, seguida de la primera y finalmente la segunda.

## 7. Imagínese que este entregable es una labor que le han solicitado en un entorno profesional, y que tiene que entregar esta documentación para comentar lo que ha descubierto (datos de entrada, rendimiento de los modelos, o cualquier descubrimiento que pueda ser importante). Comente los resultados obtenidos.

Tras haber realizado la práctica, podemos llegar a las siguientes conclusiones:

- Nuestro conjunto de datos contiene múltiples mensajes de spam y de no spam (usuarios) los cuales contienen caracteres extraños que nos dificultan la tarea. Por ello, es necesario realizar un preprocesamiento para obtener resultados lógicos
- Dicho pre-procesamiento ha mejorado mucho los resultados de las matrices tfidf.
- De los tres modelos, el mejor es el SVM en términos de exactitud. Es importante destacar que de los 3, el clasificador bayesiano es el que menos falsos positivos tiene.
- Tras obtener las matrices tfidf teniendo en cuenta las formas canónicas y eliminando los stopwords, no se puede obtener información muy concreta. Por lo general, tanto los mensajes de spam como de no spam no muestra una actitud negativo sino más neutral o incluso positiva. Prueba de ello es la aperiencia de palabras como "good", "free" o "love" en la matriz.

Para añadir mas valor, me gustaría explicar una idea que me ha surgido para mejorar los resultados. Realizando un análisis de los datos, he descubierto que tanto los mensajes spam como los que no lo son contienen una serie de caracteres extraños que nos aportan ruido a nuestras conclusiones. Una buena idea sería centrarse en los mensajes que no son spam y eliminar todos estos caracteres y números, de tal manera que nos quedemos con mensajes limpios escritos por usuarios. Esta solución que propongo sería parte del pre-procesamiento y lo haría de la siguiente forma:

1. Realizar una función que nos indique cuales son todos los caracteres únicos en nuestro corpus de documentos
2. Una vez tenemos estos caracteres en pantalla, nos fijamos en aquellos que queremos eliminar
3. Aplicamos una función que elimina estos caracteres

Pese a que este problema se podría solucionar obteniendo la forma canónica de la palabra, esto se podría ver afectado si estos caracteres se encontrasen en mitad de la palabra. Por ejemplo, si la palabra "hola" apareciese en algún mensaje como "h#ola", alomejor la función que obtuviese la forma canónica no funcionaría. Puesto que tenemos mas de 5000 documentos y no podemos ir uno a uno fijándonos en esto ya que queremos optimizar el tiempo, una buena solución sería esta que propongo.

In [19]:
data_extra=data[data['label']=='ham']
data_extra = data_extra.applymap(lambda s: s.lower() if type(s) == str else s)

lista_minusculas=list(data_extra['text'])
lista_unica=(" ".join(lista_minusculas))
lista_unica=list(set(lista_unica))

array_unica=np.array(lista_unica).T
array_unica

array(['.', 'k', '\r', 'ä', ']', '|', '+', 'q', 'ö', '4', '#', 'ò', "'",
       'l', 'n', '-', 'm', '©', '‹', '÷', 'h', 'ó', '6', 'ž', '8', 'ð',
       '´', '0', '/', 'f', 'ª', '@', '?', '*', ' ', 'è', 'e', 'å', 'b',
       'o', '=', 'w', 'a', '(', 'c', 'v', '%', '&', 'ï', 'r', '7', '¬',
       '‰', '~', 'x', ':', 'u', '^', 'õ', 'â', '1', 'ì', ',', 'y', ')',
       's', 'p', 'z', 'û', '3', '2', '£', '!', '_', '5', '[', 'd', 'ô',
       'g', '$', 'j', '"', 'i', 't', '\\', '9', ';'], dtype='<U1')

In [20]:
num_to_remove = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',' ','1','2','3','4','5','6','7','8','9']
for n in num_to_remove:
    while n in lista_unica:
        lista_unica.remove(n)

array_caracteres=np.array(lista_unica).T
array_caracteres

array(['.', '\r', 'ä', ']', '|', '+', 'ö', '#', 'ò', "'", '-', '©', '‹',
       '÷', 'ó', 'ž', 'ð', '´', '0', '/', 'ª', '@', '?', '*', 'è', 'å',
       '=', '(', '%', '&', 'ï', '¬', '‰', '~', ':', '^', 'õ', 'â', 'ì',
       ',', ')', 'û', '£', '!', '_', '[', 'ô', '$', '"', '\\', ';'],
      dtype='<U1')

En primer lugar, creo un nuevo dataframe el cual sólo contiene los mensajes de usuarios. Posteriormente, obtenemos de este nuevo dataframe todos los caracteres únicos. Finalmente, nos quedamos sólo con las letras del abecedario y los espacios en una lista y el resto de caracteres en otra. Es importante destacar que esto es simplemente un concepto inicial, y que se podría ir detallando más añadiendo símbolos como los del dinero ($,€) o los de operaciones (+,-) que pueden indicar gusto o disgusto del usuario.

#### Ejemplo

In [21]:
import re
characters = ['û', 'ð', ')', '_', '0', '-', '\r', '´', ':', '£', '÷', "'", '&',
       'ô', 'ª', '(', 'ì', '%', '\\', ',', 'è', 'ä', 'ï', '/', '"', '+',
       '@', 'ž', ';', 'õ', '=', '‹', '©', '$', 'ò', '¬', '?', '‰', '~',
       '|', 'å', '!', '#', '.', 'ö', '[', '*', ']', 'â', 'ó', '^']
for n in characters:
    while n in data_extra['text'][21]:
        data_extra['text'][21]=re.sub(n,"",data_extra['text'][21])
        
print('La frase original es:', data['text'][21])        
print('La frase modificada es:', data_extra['text'][21])

La frase original es: I‰Û÷m going to try for 2 months ha ha only joking
La frase modificada es: im going to try for 2 months ha ha only joking


Como se puede observar, la frase modificada cambia mucho y es mucho más sencilla de analizar. Si aplicamos una función la cual realice esta operación del ejemplo para todo nuestro corpus, seríamos capaces de obtener unos datos mucho más limpios que nos facilitarían a la hora de obtener resultados.